### Script para ingresar datos a base de datos remota 

In [ ]:
import mysql.connector
import pandas as pd
import requests

# Database connection parameters

host = "154.49.247.181"
database = "u506324710_b2bconn"
user = "u506324710_ivanb2b"
password = "e@6A*jI~^c"
port = 3306  # Default MySQL port (change if needed)

# Fetch the data from the JSON URL
json_url = "https://www.datos.gov.co/resource/p6dx-8zbt.json"
all_data = []
limit =100000
offset = 0
max_records = 100000

while len(all_data) < max_records:
    response = requests.get(f"{json_url}?$limit={limit}&$offset={offset}")
    data = response.json()
    if not data:
        break
    all_data.extend(data)
    offset += limit
    if len(all_data) >= max_records:
        all_data = all_data[:max_records]
        break

# Convert data to DataFrame
df = pd.DataFrame(all_data)
print("Total de registros descargados:", len(df))

# Map column names to match MySQL table structure
table_columns = [
    'entidad', 'nit_entidad', 'departamento_entidad', 'ciudad_entidad', 'ordenentidad',
    'codigo_pci', 'id_del_proceso', 'referencia_del_proceso', 'ppi', 'id_del_portafolio',
    'nombre_del_procedimiento', 'descripci_n_del_procedimiento', 'fase', 'fecha_de_publicacion_del',
    'fecha_de_ultima_publicaci', 'fecha_de_publicacion_fase_3',
    'precio_base', 'modalidad_de_contratacion', 'justificaci_n_modalidad_de', 'duracion', 'unidad_de_duracion',
    'ciudad_de_la_unidad_de', 'nombre_de_la_unidad_de', 'proveedores_invitados',
    'proveedores_invitacion_directa', 'visualizaciones_procedimiento', 'proveedores_interes',
    'respuestas_externas', 'conteo_respuestas_ofertas', 'proveedores_unicos_respuestas', 'numero_lotes',
    'estado_del_procedimiento', 'id_estado_procedimiento', 'adjudicado', 'id_adjudicacion', 'codigoproveedor',
    'departamento_proveedor', 'ciudad_proveedor', 'valor_total_adjudicacion',
    'nombre_del_adjudicador', 'nombre_del_proveedor', 'nit_del_proveedor_adjudicado',
    'codigo_principal_de_categoria', 'estado_apertura_proceso', 'tipo_de_contrato', 'subtipo_contrato',
    'categorias_adicionales', 'url_proceso', 'codigo_entidad'
]

# Ensure DataFrame has all columns, adding None for missing ones
for col in table_columns:
    if col not in df.columns:
        df[col] = None

# Convert NaN values to None for SQL compatibility
df = df.where(pd.notnull(df), None)

# Format the URL properly from the nested JSON field (if needed)
df['url_proceso'] = df['urlproceso'].apply(lambda x: x['url'] if isinstance(x, dict) else None)

# Format dates if necessary (e.g., convert strings to datetime)
df['fecha_de_publicacion_del'] = pd.to_datetime(df['fecha_de_publicacion_del'], errors='coerce')
df['fecha_de_ultima_publicaci'] = pd.to_datetime(df['fecha_de_ultima_publicaci'], errors='coerce')
df['fecha_de_publicacion_fase_3'] = pd.to_datetime(df['fecha_de_publicacion_fase_3'], errors='coerce')


# Connect to the MySQL database
try:
    conn = mysql.connector.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    cursor = conn.cursor()

    # Insert data row by row and commit after each insert
    for _, row in df[table_columns].iterrows():
        values = tuple(row)  # Convert row to tuple
        insert_query = f'''
            INSERT INTO secop_2 ({", ".join(table_columns)})
            VALUES ({", ".join(["%s"] * len(table_columns))})
        '''
        cursor.execute(insert_query, values)
        conn.commit()  # Commit after each insert

    print("Data inserted successfully into MySQL table")

except mysql.connector.Error as e:
    print(f"Error inserting data: {e}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
